In [4]:
#!{sys.executable} -m pip install matplotlib

In [209]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets

In [227]:
def filter_df(df_orig, col, arg, op='=='):
    df = df_orig.copy()
    
    if op == '==':
        df = df.loc[df[col]==arg]
    elif op == '<':
        df = df.loc[df[col]<arg]
    elif op == '>':
        df = df.loc[df[col]>arg]
    elif op == '<=':
        df = df.loc[df[col]<=arg]
    elif op == '>=':
        df = df.loc[df[col]>=arg]
    elif op == '!=':
        df = df.loc[df[col]!=arg]
    else:
        print('op type not supported!')
    
    return df

In [228]:
config_df = pd.read_csv("../cmake-build-debug/config_log.txt", sep=",", header=0)
display(config_df)
config_dict = config_df.to_dict(orient='list')


sim_df = pd.read_csv("../cmake-build-debug/sim_log.txt", sep=",", header=0)
display(sim_df.head())

sampling_df = pd.read_csv("../cmake-build-debug/sampling_log.txt", sep=",", header=0)
display(sampling_df.head())

,sim_time,rollouts,horizon,obstacle_cost,obstacle_rad_0,obstacle_rad_1,obstacle_rad_2,obstacle_rad_3,obstacle_pos_0_x,obstacle_pos_0_y,obstacle_pos_1_x,obstacle_pos_1_y,obstacle_pos_2_x,obstacle_pos_2_y,obstacle_pos_3_x,obstacle_pos_3_y,target_state_0,target_state_1,Unnamed: 18
0,100,20,10,10000,5,8,5,5,20,10,40,30,10,25,100,40,100,50,NaN


,simstep,state_0,state_1,state_2,state_3,state_4,state_5,Unnamed: 7
0,0,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,NaN
1,1,0.512821,-0.269323,1.02564,-0.538645,1.02564,-0.538645,NaN
2,2,1.363450,-0.765230,2.72690,-1.530460,1.70126,-0.991816,NaN
3,3,2.778430,-0.666374,5.55686,-1.332750,2.82996,0.197714,NaN
4,4,4.609060,-0.549322,9.21812,-1.098640,3.66126,0.234104,NaN


,simstep,step,rollout,state_0,state_1,state_2,state_3,state_4,state_5,cost_cum,Unnamed: 10
0,0,0,0,-0.368462,-0.041350,-0.736924,-0.082700,-0.736924,-0.082700,223.955,NaN
1,0,0,1,-0.281041,0.178865,-0.562082,0.357729,-0.562082,0.357729,223.779,NaN
2,0,0,2,0.434693,0.019416,0.869386,0.038833,0.869386,0.038833,223.209,NaN
3,0,0,3,-0.465428,0.029700,-0.930856,0.059400,-0.930856,0.059400,224.010,NaN
4,0,0,4,-0.492302,-0.433158,-0.984604,-0.866316,-0.984604,-0.866316,224.241,NaN


In [229]:
#df_timestep = filter_df(sampling_df, "simstep", 10)
#df_timestep = filter_df(df_timestep, "step", 3)

In [233]:
def draw_interactive(simstep, step):
    df_timestep = filter_df(sampling_df, "simstep", simstep, op='==')
    df_timestep = filter_df(df_timestep, "step", step, op='<=')
    
    fig, ax = fig, ax = plt.subplots(1, figsize=(15,10))
   
    if config_dict['obstacle_cost'][0] > 0:
        for obstacle_nr in range(0,4):
            obst_id_rad = 'obstacle_rad_'+str(obstacle_nr)
            obst_id_x = 'obstacle_pos_'+str(obstacle_nr)+'_x'
            obst_id_y = 'obstacle_pos_'+str(obstacle_nr)+'_y'
            
            ellipse_plot_0 = patches.Circle((config_dict[obst_id_x][0],config_dict[obst_id_y][0]), 
                                     radius=config_dict[obst_id_rad][0])
      
            ax.add_patch(ellipse_plot_0)
    
    ax.plot(sim_df[['state_0']], sim_df[['state_1']])
    ax.scatter(df_timestep[['state_0']], df_timestep[['state_1']], s=1)
    ax.scatter(sim_df[['state_0']].iloc[simstep], sim_df[['state_1']].iloc[simstep],s=60)


    padding = 10;
    ax.set_xlim(sim_df[['state_0']].min()[0]-padding, sim_df[['state_0']].max()[0]+padding)
    ax.set_ylim(sim_df[['state_1']].min()[0]-padding, sim_df[['state_1']].max()[0]+padding)
    plt.show()
    
widgets.interactive(draw_interactive, 
                    simstep = widgets.IntSlider(value=0, min=0, max=sim_df.shape[0]-2, step=1), 
                    step= widgets.IntSlider(value=19, min=0, max=19))

interactive(children=(IntSlider(value=0, description='simstep', max=99), IntSlider(value=19, description='step…

In [225]:
sim_df[['state_0']].min()[0]

0.0